In [1]:
import pandas as pd
import os
pd.set_option("max_columns",None)
import shutil
from xml.dom.minidom import parse
import xml.dom.minidom
from collections import Counter

In [2]:
work_path=os.getcwd()

### 解析表格 文件名，列名，行数

In [3]:
def get_table_Infor(Table):
    Table_dic={}
    columns=Table.getElementsByTagName("Column")
    for column in columns:
        position=column.attributes["position"].value
        name=column.getElementsByTagName("Name")[0].childNodes[0].nodeValue.strip()
        Table_dic[position]=name
    table=pd.DataFrame.from_dict(Table_dic,orient="index").reset_index()
    table.columns=["Position","Columns"]
    table_file_name=Table.getElementsByTagName("External-Data")[0].childNodes[0].nodeValue.strip()
    table_rows=Table.getElementsByTagName("External-Data")[0].attributes["rows"].value
    return table,table_file_name,table_rows

### 解析项目参与人

In [4]:
def get_contributor_infor(contributor):
    contributor_dic={}
    contributor_dic["iid"]=contributor.attributes["iid"].value
    for tag in ["Email","Phone","Fax","Laboratory","Department"]:
        tag_list=contributor.getElementsByTagName(tag)
        for t in tag_list:
            for c in t.childNodes:
                contributor_dic[tag]=c.nodeValue
    for tag in ["Person","Address"]:
        tag_list=contributor.getElementsByTagName(tag)
        for t in tag_list:
            contributor_dic[tag]=""
            for c in t.childNodes:
                for n in c.childNodes:
                    #if contributor_dic[tag]=="":
                    contributor_dic[tag]+=c.tagName+":"+n.wholeText+"; "
    return pd.DataFrame.from_dict(contributor_dic,orient="index").T
def get_contributors(contributors):
    R=[]
    for contributor in contributors:
        contributor_infor=get_contributor_infor(contributor)
        R.append(contributor_infor)
    R=pd.concat(R).reset_index(drop=True).fillna("")
    return R

### 解析 实验平台信息

In [5]:
def get_Platform_Infor(Platform):
    Platform_dic={}
    Platform_dic["iid"]=Platform.attributes["iid"].value
    for tag in ["Title","Accession","Technology","Organism","Distribution",
                "Manufacturer","Manufacture-Protocol","External-Data"]:
        tag_list=Platform.getElementsByTagName(tag)
        for t in tag_list:
            for c in t.childNodes:
                Platform_dic[tag]=c.nodeValue.strip().strip("\n")
    for tag in["Web-Link"]:
        tag_list=Platform.getElementsByTagName(tag)
        Platform_dic[tag]=""
        for t in tag_list:
            for c in t.childNodes:
                Platform_dic[tag]+=c.wholeText+"; "
    for tag in["Description"]:
        tag_list=Platform.getElementsByTagName(tag)
        Platform_dic[tag]=""
        for t in tag_list[:1]:
            for c in t.childNodes:
                Platform_dic[tag]+=c.wholeText.strip().strip("\n")
    for tag in ["Status"]:
        tag_list=Platform.getElementsByTagName(tag)
        for t in tag_list:
            Platform_dic[tag]=""
            for c in t.childNodes:
                for n in c.childNodes:
                    Platform_dic[tag]+=c.tagName+":"+n.wholeText+"; "
    platform = pd.DataFrame.from_dict(Platform_dic,orient="index").T
    return platform
def get_Platforms(Platforms):
    R=[]
    for Platform in Platforms:
        platform= get_Platform_Infor(Platform)
        R.append(platform)
    R=pd.concat(R).reset_index(drop=True).fillna("")
    return R

### 解析样本信息

In [6]:
def get_Samples_Infor(Sample):
    Sample_dic={}
    Sample_dic["iid"]=Sample.attributes["iid"].value
    for tag in ["Title","Accession","Type","Channel-Count","Hybridization-Protocol","Scan-Protocol",
                "Description","Data-Processing","Supplementary-Data"]:
        tag_list=Sample.getElementsByTagName(tag)
        for t in tag_list:
            for c in t.childNodes:
                Sample_dic[tag]=c.nodeValue.strip().strip("\n")
    for tag in["Platform-Ref","Contact-Ref"]:
        tag_list=Sample.getElementsByTagName(tag)
        for t in tag_list:
            Sample_dic[tag]=t.attributes["ref"].value
    for tag in["Channel"]:
        Channels=Sample.getElementsByTagName(tag)
        for Channel in Channels:
            pos=Channel.attributes["position"].value
            for sub_tag in ["Source","Organism","Characteristics","Treatment-Protocol","Growth-Protocol",
                    "Molecule","Extract-Protocol","Label","Label-Protocol",]:
                tag_list=Channel.getElementsByTagName(sub_tag)
                for t in tag_list:
                    for c in t.childNodes:
                        if t.attributes:
                            if "tag" in t.attributes:
                                if sub_tag in Sample_dic:
                                    Sample_dic[sub_tag]+=t.attributes["tag"].value+":"+c.nodeValue.strip()+"; "
                                else:
                                    Sample_dic[sub_tag]=t.attributes["tag"].value+":"+c.nodeValue.strip()+"; "
                            elif "taxid" in t.attributes:
                                if sub_tag in Sample_dic:
                                    Sample_dic[sub_tag]+=t.attributes["taxid"].value+":"+c.nodeValue.strip()+"; "
                                else:
                                    Sample_dic[sub_tag]=t.attributes["taxid"].value+":"+c.nodeValue.strip()+"; "
                            else:
                                Sample_dic[tag+"-"+pos+":"+sub_tag]=c.nodeValue.strip().strip("\n")
                        else:
                            Sample_dic[tag+"-"+pos+":"+sub_tag]=c.nodeValue.strip().strip("\n")
    for tag in ["Status"]:
        tag_list=Sample.getElementsByTagName(tag)
        for t in tag_list:
            Sample_dic[tag]=""
            for c in t.childNodes:
                for n in c.childNodes:
                        Sample_dic[tag]+=c.tagName+":"+n.wholeText+"; "
    sample = pd.DataFrame.from_dict(Sample_dic,orient="index").T
    return sample

def get_Samples(Samples):
    R=[]
    for Sample in Samples:
        sample=get_Samples_Infor(Sample)
        R.append(sample)
    R=pd.concat(R).reset_index(drop=True).fillna("")
    return R

### 解析数据集信息

In [7]:
#Series=collection.getElementsByTagName("Series")
def get_Series_Infor(Series):
    Series_dic={}
    Series_dic["iid"]=Series.attributes["iid"].value
    for tag in ["Title","Accession","Pubmed-ID","Summary","Overall-Design",
                "Description","Data-Processing","Supplementary-Data"]:
        tag_list=Series.getElementsByTagName(tag)
        for t in tag_list:
            for c in t.childNodes:
                Series_dic[tag]=c.nodeValue.strip().strip("\n")
    for tag in["Contributor-Ref","Contact-Ref","Sample-Ref"]:
        tag_list=Series.getElementsByTagName(tag)
        Series_dic[tag]=""
        for t in tag_list:
            Series_dic[tag]+=t.attributes["ref"].value+"; "
    for tag in["Type"]:
        Types_list=Series.getElementsByTagName(tag)
        Series_dic[tag]=""
        for Type in Types_list:
                for c in Type.childNodes:
                    Series_dic[tag]+=c.nodeValue.strip().strip("\n")+"; "
    for tag in ["Status"]:
        tag_list=Series.getElementsByTagName(tag)
        for t in tag_list:
            Series_dic[tag]=""
            for c in t.childNodes:
                for n in c.childNodes:
                        Series_dic[tag]+=c.tagName+":"+n.wholeText+"; "
    series = pd.DataFrame.from_dict(Series_dic,orient="index").T
    return series

### 解析

In [8]:
Tables_Infor=[]
GEO_Infor=[]
Authors=[]
Platform_Infor=[]
Sample_Infor=[]

In [10]:
file_path="../test_sample/"

xml_files=os.listdir(file_path)
count=0
for file in xml_files:
    if file.endswith("_family.xml"):
        geo_id=file.split("_")[0]
        count+=1
        if count%100==0:
            print(count)
        try:
            DOMTree = xml.dom.minidom.parse(file_path+file)
            collection = DOMTree.documentElement
            Tables=collection.getElementsByTagName("Data-Table")
            #print("表格数：",len(Tables))
            for Table in Tables:
                table,table_file_name,table_rows=get_table_Infor(Table)
                table["file_name"]=table_file_name
                table["rows"]=table_rows
                #table.to_excel(table_path+table_file_name.replace(".txt","_"+table_rows+".xlsx"),index=False)
                Tables_Infor.append(table)
            contributors=collection.getElementsByTagName("Contributor")
            contributors= get_contributors(contributors)
            contributors["acc"]=geo_id
            #print("参与人员数：",contributors.shape[0])
            Authors.append(contributors)
            #contributors.to_excel(contributors_path+geo_id+".xlsx",index=False)
            Platforms=collection.getElementsByTagName("Platform")
            try:
                platforms=get_Platforms(Platforms)
                #print("实验平台数：",platforms.shape[0])
                platforms["acc"]=geo_id
                Platform_Infor.append(platforms)
                #platforms.to_excel(platforms_path+geo_id+".xlsx",index=False)
            except:
                pass
            Samples=collection.getElementsByTagName("Sample")
            if len(Samples)>0:
                samples=get_Samples(Samples)
                #print("样本数：",samples.shape[0])
                samples["acc"]=geo_id
                Sample_Infor.append(samples)
                #samples.to_excel(samples_path+geo_id+".xlsx",index=False)
            Series=collection.getElementsByTagName("Series")
            series=get_Series_Infor(Series[0])
            GEO_Infor.append(series)
        except Exception as e:
            print(e,"\n")
            print(file)

3


In [15]:
GEO_Infor[0]

,iid,Title,Accession,Summary,Overall-Design,Supplementary-Data,Contributor-Ref,Contact-Ref,Sample-Ref,Type,Status
0,GSE153424,Cell types and clonal relations in the mouse b...,GSE153424,We report TREX for simultaneous lineage tracki...,Lentiviral delivery of a transcribed barcode i...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE153nn...,contrib1; contrib3; contrib4; contrib5; contri...,contrib1;,GSM4644058; GSM4644059; GSM4644060; GSM4644061...,Expression profiling by high throughput sequen...,Submission-Date:2020-06-28; Release-Date:2020-...


In [11]:
today_date="2022-01-13"

In [17]:
result_path="../Result/"
os.mkdir(result_path+"GEO")
if Tables_Infor!=[]:
    Tables_Infor=pd.concat(Tables_Infor).reset_index(drop=True)
    print(Tables_Infor.shape)
    Tables_Infor.to_csv(result_path+"GEO/"+"GEO_Tables_headers_infor_"+today_date+".csv",index=False)

In [18]:
#Authors.append(contributors)
Authors=pd.concat(Authors).reset_index(drop=True).fillna("")
def get_author_name(x):
    values= x.split("; ")
    name=[]
    for v in values:
        if v!="":
            name.append(v.split(":")[1])
    return " ".join(name)
Authors["Name"]=Authors["Person"].apply(get_author_name)
#Authors.to_csv("E:\\GEO dataset\\GEO Result\\GEO_Authors_infor_"+today_date+".csv",index=False)

In [34]:
def get_dataset_author_name(x):
    first=""
    middle=""
    last=""
    names=x.split("; ")
    for name in names:
        if name.startswith("First:"):
            first=name.split(":")[1].strip(";")
        if name.startswith("Last:"):
            last=name.split(":")[1].strip(";")
        if name.startswith("Middle:"):
            middle=name.split(":")[1].strip(";")
    if middle!="":
        return last+", "+first+" "+middle
    else:
        return last+", "+first
Authors["Name"]=Authors["Person"].apply(get_dataset_author_name)

In [36]:
au_dic={}
for p,p_df in Authors.groupby("Name"):
    au_dic[p]=" | ".join(list(p_df["acc"].unique()))
#del Authors["acc"]
#Authors=Authors.drop_duplicates(["Name","Email","Laboratory"]).reset_index(drop=True)
Authors["Author_Datasets"]=Authors["Name"].apply(lambda x:au_dic[x])
print(Authors.shape)
#del Authors["iid"]
Authors.to_csv(result_path+"GEO/"+"GEO_Authors_infor_"+today_date+".csv",index=False)
Authors.head(1)

(9, 9)


,iid,Email,Laboratory,Department,Person,Address,acc,Name,Author_Datasets
0,contrib1,michael.ratz@ki.se,Frisén,CMB,First:Michael; Last:Ratz;,Line:Solnavägen 9; City:Solna; Postal-Code:171...,GSE153424,"Ratz, Michael",GSE153424


In [20]:
Platform_Infor=pd.concat(Platform_Infor).reset_index(drop=True).fillna("")
print(Platform_Infor.shape)
Platform_Infor.to_csv(result_path+"GEO/"+"GEO_Platform_infor_"+today_date+".csv",index=False)
Platform_Infor.head(1)

(4, 11)


,iid,Title,Accession,Technology,Organism,Distribution,Manufacture-Protocol,Web-Link,Description,Status,acc
0,GPL16085,Illumina MiSeq (Escherichia coli),GPL16085,high-throughput sequencing,Escherichia coli,virtual,,,,Submission-Date:2012-09-20; Release-Date:2012-...,GSE153424


In [21]:
Samples=pd.concat(Sample_Infor).reset_index(drop=True).fillna("")
samples_dic={}
for p,p_df in Samples.groupby("iid"):
    samples_dic[p]=" | ".join(list(p_df["acc"].unique()))
Samples["Sample_Datasets"]=Samples["iid"].apply(lambda x:samples_dic[x])
print(Samples.shape)
Samples.to_csv(result_path+"GEO/"+"GEO_Sample_Infor_"+today_date+".csv",index=False)
Samples.head(1)

(38, 19)
(38, 20)


,iid,Title,Accession,Type,Channel-Count,Description,Data-Processing,Supplementary-Data,Platform-Ref,Contact-Ref,Channel-1:Source,Organism,Characteristics,Channel-1:Treatment-Protocol,Channel-1:Growth-Protocol,Channel-1:Molecule,Channel-1:Extract-Protocol,Status,acc,Sample_Datasets
0,GSM4644058,brain1_cx,GSM4644058,SRA,1,gene expression library,Chromium: Sequencing reads were processed with...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4644...,GPL24247,contrib1,brain cells,10090:Mus musculus;,region:cortex; age:P11; treatment:barcoded; re...,barcoding of progenitor cells was achieved via...,mice were kept in standard housing conditions,polyA RNA,Brains were dissected and single cell dissocia...,Submission-Date:2020-06-28; Release-Date:2020-...,GSE153424,GSE153424


In [23]:
author_dic={}
for acc,acc_df in Authors.groupby("acc"):
    acc_df=acc_df.reset_index(drop=True)
    author_dic[acc]={}
    for i in range(acc_df.shape[0]):
        author_dic[acc][acc_df.loc[i,"iid"]]=acc_df.loc[i,"Name"]
email_dic={}
for acc,acc_df in Authors.groupby("acc"):
    acc_df=acc_df.reset_index(drop=True)
    email_dic[acc]={}
    for i in range(acc_df.shape[0]):
        if acc_df.loc[i,"Email"]!="":
            email_dic[acc][acc_df.loc[i,"Name"]]=acc_df.loc[i,"Email"]
platform_dic={}
for acc,acc_df in Platform_Infor.groupby("acc"):
    platform_dic[acc]=" | ".join(list(acc_df["iid"].unique()))

In [24]:
GEO_Infor=pd.concat(GEO_Infor).reset_index(drop=True).fillna("")
print(GEO_Infor.shape)

(1, 11)


In [25]:
GEO_Infor["platform"]=GEO_Infor["iid"].apply(lambda x:platform_dic[x] if x in platform_dic else 0)
GEO_Infor["contributor"]=""
GEO_Infor["contact_author"]=""
GEO_Infor["contact_email"]=""
for i in range(GEO_Infor.shape[0]):
    acc=GEO_Infor.loc[i,"iid"]
    
    if GEO_Infor.loc[i,"Contact-Ref"].strip("; ")!="":
        contact=author_dic[acc][GEO_Infor.loc[i,"Contact-Ref"].strip("; ")]
        GEO_Infor.loc[i,"contact_author"]=contact
        if contact in email_dic[acc]:
            GEO_Infor.loc[i,"contact_email"]=email_dic[acc][contact]
    contributor=[]
    cons=GEO_Infor.loc[i,"Contributor-Ref"].split("; ")
    for con in cons:
        if con in author_dic[acc]:
            contributor.append(author_dic[acc][con])
    GEO_Infor.loc[i,"contributor"]=" | ".join(contributor)
GEO_Infor.head(1)

,iid,Title,Accession,Summary,Overall-Design,Supplementary-Data,Contributor-Ref,Contact-Ref,Sample-Ref,Type,Status,platform,contributor,contact_author,contact_email
0,GSE153424,Cell types and clonal relations in the mouse b...,GSE153424,We report TREX for simultaneous lineage tracki...,Lentiviral delivery of a transcribed barcode i...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE153nn...,contrib1; contrib3; contrib4; contrib5; contri...,contrib1;,GSM4644058; GSM4644059; GSM4644060; GSM4644061...,Expression profiling by high throughput sequen...,Submission-Date:2020-06-28; Release-Date:2020-...,GPL16085 | GPL16417 | GPL21626 | GPL24247,Michael Ratz | Leonie von Berlin | Ludvig Lars...,Michael Ratz,michael.ratz@ki.se


In [26]:
species_dic={}
cell_type_dic={}
starin_dic={}
tissue_dic={}
develop_dic={}
sex_dic={}
disease_dic={}
age_dic={}
char_dic={}
for acc,acc_df in Samples.groupby("acc"):
    species_dic[acc]=" | ".join(list(acc_df["Organism"].unique()))
    chars=list(set("; ".join(acc_df[acc_df["Characteristics"]!=""]["Characteristics"].tolist()).split("; ")))
    char_dic[acc]=" | ".join(chars)
    cell_type_dic[acc]=[]
    for char in chars:
        if "cell" in char.split(":")[0].lower():
            if acc not in cell_type_dic:
                cell_type_dic[acc]=[]
            cell_type_dic[acc].append(char)
        if "starin" ==char.split(":")[0]:
            starin_dic[acc]=char.split(":")[1]
        if "tissue" in char.split(":")[0].lower():
            if acc not in tissue_dic:
                tissue_dic[acc]=[]
            tissue_dic[acc].append(char)
        if "develop" in char.split(":")[0].lower():
            if acc not in develop_dic:
                develop_dic[acc]=[]
            develop_dic[acc].append(char)
        if "sex" in char.split(":")[0].lower():
            if acc not in sex_dic:
                sex_dic[acc]=[]
            sex_dic[acc].append(char)
        if "disease" in char.split(":")[0].lower():
            if acc not in disease_dic:
                disease_dic[acc]=[]
            disease_dic[acc].append(char)
        if "age" in char.split(":")[0].lower():
            if acc not in age_dic:
                age_dic[acc]=[]
            age_dic[acc].append(char)
    if acc in cell_type_dic:
        cell_type_dic[acc]=" | ".join(cell_type_dic[acc]).replace("cell type:","").replace("Cell type:","")
    if acc in tissue_dic:
        tissue_dic[acc]=" | ".join(tissue_dic[acc]).replace("tissue:","").replace("Tissue:","")
    if acc in develop_dic:
        develop_dic[acc]=" | ".join(develop_dic[acc]).replace("development stage:","").replace("Development stage:","")
    if acc in sex_dic:
        sex_dic[acc]=" | ".join(sex_dic[acc]).replace("sex:","").replace("Sex:","")
    if acc in disease_dic:
        disease_dic[acc]=" | ".join(disease_dic[acc]).replace("disease:","").replace("Disease:","")
    if acc in age_dic:
        age_dic[acc]=" | ".join(age_dic[acc]).replace("age:","").replace("Age:","")

In [27]:
GEO_Infor["species"]=GEO_Infor["iid"].apply(lambda x:species_dic[x] if x in species_dic else "")
GEO_Infor["cell_types"]=GEO_Infor["iid"].apply(lambda x:cell_type_dic[x] if x in cell_type_dic else "")
GEO_Infor["tissue"]=GEO_Infor["iid"].apply(lambda x:tissue_dic[x] if x in tissue_dic else "")
GEO_Infor["development_stage"]=GEO_Infor["iid"].apply(lambda x:develop_dic[x] if x in develop_dic else "")
GEO_Infor["sex"]=GEO_Infor["iid"].apply(lambda x:sex_dic[x] if x in sex_dic else "")
GEO_Infor["disease"]=GEO_Infor["iid"].apply(lambda x:disease_dic[x] if x in disease_dic else "")
GEO_Infor["age"]=GEO_Infor["iid"].apply(lambda x:age_dic[x] if x in age_dic else "")
GEO_Infor["Characteristics"]=GEO_Infor["iid"].apply(lambda x:char_dic[x] if x in char_dic else "")

In [28]:
GEO_Infor=GEO_Infor.rename(columns={"Title":'title',
                   "Summary":"summary",
                   "Overall-Design":"overall_design",
                    "contributor":"contributors",
                   "contact_email":"contact",
                   "Accession":"accessions",
                    "Pubmed-ID":"pmid",
                   })

In [29]:
def get_submission_date(x):
    values=x.split("; ")
    for v in values:
        if "Submission" in v:
            return v.split(":")[1]
    return ""
GEO_Infor["submission_date"]=GEO_Infor["Status"].apply(get_submission_date)
def get_update_date(x):
    values=x.split("; ")
    for v in values:
        if "Last-Update" in v:
            return v.split(":")[1]
    return ""
GEO_Infor["last_update_date"]=GEO_Infor["Status"].apply(get_update_date)

In [31]:
GEO_Infor.to_csv(result_path+"GEO/"+"GEO_Dataset_Infor_"+str(GEO_Infor.shape[0])+"_"+today_date+".csv",index=False)